In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf


from matplotlib import pyplot as plt
from scipy.misc import imread, imresize

if tf.__version__ != '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.0!')

%matplotlib inline

In [ ]:
BASE_DIR = '/home/wenfeng/all-files/skin-lesion-seg-v2'

In [ ]:
import crf
import inputs
import my_utils

from sklearn.model_selection import KFold

In [ ]:
config = my_utils.load_config(os.path.join(BASE_DIR, 'config.json'))
image_config = my_utils.load_config(os.path.join(BASE_DIR, 'image_config.json'))

In [ ]:
class RestoredModel:
    def __init__(self, ckpt_file):
        self.graph = tf.Graph()
        with self.graph.as_default() as g:
            with tf.device('/cpu'):
                od_graph_def = tf.GraphDef()
                with tf.gfile.GFile(ckpt_file, 'rb') as fid:
                    sg = fid.read()
                    od_graph_def.ParseFromString(sg)
                    tf.import_graph_def(od_graph_def, name='')

                self.image_ph = g.get_tensor_by_name('image_tensor:0')
                self.bboxes = g.get_tensor_by_name('detection_boxes:0')
                self.scores = g.get_tensor_by_name('detection_scores:0')
                self.n_bboxes = g.get_tensor_by_name('num_detections:0')
    
    def inference_box(self, image):
        sess = tf.get_default_session()
        image = image[None] if len(image.shape) == 3 else image
        return sess.run(self.bboxes, feed_dict={self.image_ph: image})[0, 0]

In [ ]:
PATH_TO_CKPT = os.path.join(BASE_DIR, 'training/output_inference_graph.pb/frozen_inference_graph.pb')
mm = RestoredModel(PATH_TO_CKPT)

In [ ]:
dermquest_listing = inputs.get_image_list(config['data_dir'], 'dermquest')

In [ ]:
len(dermquest_listing[0])

In [ ]:
data = dermquest_listing[0] + dermquest_listing[1]

In [ ]:
kf = KFold(image_config['n_folds'], shuffle=True, random_state=image_config['split_seed'])

train_idxes, test_idxes = list(kf.split(data))[image_config['k']]
train_data = [data[idx] for idx in train_idxes]
test_data = [data[idx] for idx in test_idxes]

In [ ]:
len(train_data), len(test_data)

In [ ]:
def bbox_xy_to_tlwh(x, size):
    """
    Change bounding-box prediction(ymin, xmin, ymax, xmax) back to its size. e.g.(0.1, 0.1, 0.8, 0.8)
    
    Return:
        result: tuple, four integers giving(top, left, height, width)
    """
    h, w = size
    ymin, xmin, ymax, xmax = x
    top = int(ymin * h)
    left = int(xmin * w)
    height = int((ymax - ymin) * h)
    width = int((xmax - xmin) * w)
    return top, left, height, width

In [ ]:
path_base = test_data[20]
with mm.graph.as_default() as g:
    with tf.Session(graph=g, config=tf.ConfigProto(device_count={'GPU': 0})):
        image = imread(path_base + '_orig.jpg')
        label = imread(path_base + '_contour.png')
        label[label == 255] = 1
        bbox_gt = my_utils.calc_bbox(label)
        
        plt.figure(figsize=(20, 10))
        plt.subplot(211)
        bbox_pred = mm.inference_box(image)
        plt.imshow(image)
        
        top, left, height, width = bbox_xy_to_tlwh(bbox_pred, image.shape[:2])
        plt.gca().add_patch(plt.Rectangle((left, top), width, height, alpha=0.2, color='b'))
        
        top, left, height, width = bbox_gt
        plt.gca().add_patch(plt.Rectangle((left, top), width, height, alpha=0.2, color='r'))
        
        
        plt.subplot(212)
        label[label == 0] = 255
        top, left, height, width = bbox_xy_to_tlwh(bbox_pred, image.shape[:2])
        plt.gca().add_patch(plt.Rectangle((left, top), width, height, alpha=0.2, color='b'))

        top, left, height, width = bbox_gt
        plt.gca().add_patch(plt.Rectangle((left, top), width, height, alpha=0.2, color='r'))
        
        plt.imshow(label, cmap='gray')
        
        plt.show()
        print(my_utils.calc_bbox_iou(bbox_gt, bbox_xy_to_tlwh(bbox_pred, image.shape[:2])))

In [ ]:
path_base = test_data[1]
with mm.graph.as_default() as g:
    with tf.Session(graph=g, config=tf.ConfigProto(device_count={'GPU': 0})):
        for i, base in enumerate(test_data):
            image, label, bbox_gt = load_one_example(base, size=(600, 400))

            bbox_pred = mm.inference_box(image)
            iou_i = my_utils.calc_bbox_iou(bbox_gt, bbox_xy_to_tlwh(bbox_pred, image.shape[:2]))
#             if iou_i < 0.6:
#                 print(i, iou_i, '----------->')
#             else:
#                 print(i, iou_i)

In [ ]:
i

In [ ]:
with mm.graph.as_default() as g:
    result = {
        'TP': 0,
        'TN': 0,
        'FP': 0,
        'FN': 0
    }
    def update_dict(target, to_update):
        for key in to_update:
            target[key] += to_update[key]
    with tf.Session(graph=g, config=tf.ConfigProto(device_count={'GPU': 0})):
        for i, base in enumerate(train_data):
            image, label, bbox_gt = load_one_example(base, size=(600, 400))
            
            result_i, _ = inference_one_image(mm, image, label, verbose=(i % 10 == 0), times=3, gt_prob=0.51)
            
            update_dict(result, result_i)
        result.update(my_utils.metric_many_from_counter(result))
        print(result)

In [ ]:
def inference_one_image(net, image, label, bbox_gt=None, verbose=True, times=1, gt_prob=0.8):
    if type(image) == str:
        image = imread(image)
    if verbose:
        print('Processing image with shape%s' % (image.shape,))
    bbox_pred = net.inference_box(image)
    bbox_pred = bbox_xy_to_tlwh(bbox_pred, size=image.shape[:2])

    prediction = crf.crf_from_bbox(image, bbox=bbox_pred, gt_prob=gt_prob)
    if times > 1:
        for i in range(times - 1):
            unary = crf.get_unary_term(prediction, unary_from='label', n_classes=2, gt_prob=gt_prob)
            prediction = crf.crf_post_process(image, unary)

    result = my_utils.count_many(prediction, label)
    prediction = {
        'label': prediction,
        'bbox': bbox_pred,
    }
    if bbox_gt is not None:
        prediction['IoU'] = my_utils.calc_bbox_iou(bbox_pred, bbox_gt)
    return result, prediction

In [ ]:
def load_one_example(base, size=None):
    image = imread(base + '_orig.jpg')
    label = imread(base + '_contour.png')
    label[label == 255] = 1
    bbox = my_utils.calc_bbox(label)
    if size:
        image = imresize(image, size=size)
        label = imresize(label, size=size, interp='nearest')
    return image, label, bbox

In [ ]:
def show_one_result(image, label, label_pred, bbox_gt, bbox_pred):
    plt.figure(figsize=(30, 20))
    plt.subplot(311)
    plt.imshow(image)
    top, left, height, width = bbox_gt
    plt.gca().add_patch(plt.Rectangle((left, top), width, height, alpha=0.2, color='b'))

    top, left, height, width = bbox_pred
    plt.gca().add_patch(plt.Rectangle((left, top), width, height, alpha=0.2, color='r'))
    
    plt.subplot(312)
    plt.imshow(label_pred, cmap='gray')
    
    plt.subplot(313)
    plt.imshow(label, cmap='gray')
    plt.show()

In [ ]:
image, label, bbox_gt = load_one_example(test_data[7])
result, prediction = inference_one_image(mm, image, label, bbox_gt, times=3, gt_prob=0.51)
print(result)
label_pred, bbox_pred = prediction['label'], prediction['bbox']
show_one_result(image, label, label_pred, bbox_gt, bbox_pred)

In [ ]:
label_pred.shape